In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [2]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [3]:
base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [4]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [5]:
# Comdty data
cindex_path = os.path.join(data_path, 'fut1return-com.csv')
cindex2_path = os.path.join(data_path, 'BCOM.csv')

cindex = pd.read_csv(cindex_path, header=0, index_col=0, parse_dates=True)
cindex2 = pd.read_csv(cindex2_path, header=0, index_col=0, parse_dates=True)

CRet = cindex.pct_change(1).iloc[1:]
CRet2 = cindex2.pct_change(1).iloc[1:]
Cindex = (1. + CRet).cumprod()

In [14]:
class CVA2(CommodityStrategy):
    def __init__(self, strategy_name, asset_type):
        super().__init__(strategy_name=strategy_name, asset_type=asset_type)
        self.commopos = None

    def load_strategy_data(self, table='bloom', origin='commo pos'):
        self._connect_database()
        source = table.upper()
        metadata = sqlalchemy.MetaData(bind=self.engine)
        table = sqlalchemy.Table('GRP_{}'.format(source), metadata, autoload=True)
        query = "select * from info_bloom"
        info = self.engine.execute(query)
        rows = [row for row in info]
        columns = info.keys()
        info = pd.DataFrame(rows, columns=columns)
        data_info = info[info.origin == origin].set_index('ticker')
        query = sqlalchemy.select('*').where(table.c.ticker.in_(data_info.index.str.upper()))
        db_data = pd.read_sql(query, self.engine)
        db_data = db_data.pivot_table('value', 'tdate', 'ticker')
        grp = pd.unique(data_info.grp_ticker)

        commo = []
        for g in grp:
            tickers_ = data_info[data_info.grp_ticker == g].index

            for ticker in tickers_:
                if ticker.split(' ')[0][-2:].upper() == 'CN':
                    ticker1 = ticker
                if ticker.split(' ')[0][-2:].upper() == 'CS':
                    ticker2 = ticker

            k = db_data[ticker1.upper()] / db_data[ticker2.upper()]
            commo.append(k)

        commo = pd.concat(commo, 1)
        commo.columns = grp
        commo.index = pd.to_datetime(commo.index)
        commo.index.name = 'tdate'
        self.commopos = commo.shift(1)

    def calculate_signal(self, CS=0.35, nopos=0.4, minobs1=12):
        """

        :param cs_num: Percent of positions for Cross Sectional Signal
        :param no_pos: no position zone in the middle of two extreme
        :param lookback_period: lookback period for calculating stdev
        :return:
        """
        self.logger.info('[STEP 3] CACULATE SIGNAL')

        for i in range(2):
            if i == 0:
                cret = self.ret[CommodityStrategy.BLOOM_COMMODITY_WEATHER_GROUP]
            if i == 1:
                cret = self.ret[CommodityStrategy.BLOOM_COMMODITY_NOTWEATHER_GROUP]

            compos1 = self.commopos[cret.columns]
            compos1 = compos1.iloc[
                compos1.reset_index().groupby(compos1.index.to_period('M'))[compos1.index.name].idxmax()]
            compos1 = compos1["1998":]
            Zscore = (compos1.iloc[minobs1:] - compos1.rolling(minobs1).mean()) / (compos1.rolling(minobs1).std())
            Zscore = Zscore.iloc[minobs1:]
            RV = -Zscore

            pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
            RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
            RVrank = RVrank.iloc[minobs1 - 1:, ]

            # 5. Long Short
            truecount = (RVrank.notnull().sum(axis=1) * CS).apply(round)
            tiebreaker = RVrank.rolling(5).mean().fillna(0) * 0.0000001

            # 1. Cross sectional
            CSRV = (RVrank + tiebreaker).rank(axis=1, method='first')  # Short
            CSRV1 = (-1 * RVrank - 1 * tiebreaker).rank(axis=1, method='first')  # Long
            
            CSRVpos = CSRV * 0
            CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
            CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1
            CSRV = CSRVpos
            CSRV.fillna(0, inplace=True)

            # 2. Time Series
            TSRV = RVrank.fillna(0) * 0
            TSRV[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
            TSRV[RVrank < (1 - nopos) / 2] = -1  # Short
            
            CSRV = CSRV * 1/2
            TSRV = TSRV * 1/2

            if i == 0:
                TSRVrun1 = TSRV
                CSRVrun1 = CSRV
            else:
                TSRVrun2 = TSRV
                CSRVrun2 = CSRV

        TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
        CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

        TSRV = TSRV[self.ret.columns]
        CSRV = CSRV[self.ret.columns]

        self.TSRV = TSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')
        self.CSRV = CSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')

        # Align dates with each other
        if self.TSRV.index[0] > self.CSRV.index[0]:
            self.CSRV = self.CSRV.loc[self.TSRV.index[0]:]
        else:
            self.TSRV = self.TSRV.loc[self.CSRV.index[0]:]

In [15]:
cva2 = CVA2(strategy_name="CVA2", asset_type="COMMODITY")
# cva2.load_index_and_return(from_db=False, save_file=False)
cva2.index = Cindex.copy()
cva2.ret = CRet.copy()

cva2.set_rebalance_period(freq='month')
# cva2.load_strategy_data(table='bloom', origin='commo pos')

commopos = pd.read_csv(os.path.join(data_path, "commo pos.csv"), index_col=0, parse_dates=True)
commopos.index.name = 'tdate'
commopos.columns.name = 'ticker'

cva2.commopos = commopos
cva2.calculate_signal(CS=0.35, nopos=0.4, minobs1=12)

2019-12-20 16:09:50,593 - CVA2 - INFO - [STEP 0] START LOGGING CVA2
2019-12-20 16:09:50,593 - CVA2 - INFO - [STEP 0] START LOGGING CVA2
2019-12-20 16:09:50,595 - CVA2 - INFO - [STEP 2] SET REBALANCE PERIOD
2019-12-20 16:09:50,595 - CVA2 - INFO - [STEP 2] SET REBALANCE PERIOD
2019-12-20 16:09:50,608 - CVA2 - INFO - [STEP 3] CACULATE SIGNAL
2019-12-20 16:09:50,608 - CVA2 - INFO - [STEP 3] CACULATE SIGNAL
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation 

In [16]:
cva2_TSRV = pd.read_csv('../past/cva2_tsrv.csv', index_col=[0])
cva2_CSRV = pd.read_csv('../past/cva2_csrv.csv', index_col=[0])

cva2_TSRV.index = pd.to_datetime(cva2_TSRV.index)
cva2_CSRV.index = pd.to_datetime(cva2_CSRV.index)

In [18]:
TSRV_DIFF = (cva2_TSRV - cva2.TSRV)

In [19]:
TSRV_DIFF[np.abs(TSRV_DIFF) > 0.01].count()

GC       0
CL     131
NG      23
HG      45
C       42
S       21
SI      67
SB      44
XBW      0
SM      66
BO     152
W       42
KC      61
CT      21
dtype: int64

In [23]:
i = 0

In [26]:
ret = CRet
index = Cindex

In [28]:
minobs1 = 12

In [40]:
if i == 0:
    cret = ret[CommodityStrategy.BLOOM_COMMODITY_WEATHER_GROUP]
if i == 1:
    cret = ret[CommodityStrategy.BLOOM_COMMODITY_NOTWEATHER_GROUP]

compos1 = commopos[cret.columns]
compos1 = compos1.iloc[
    compos1.reset_index().groupby(compos1.index.to_period('M'))[compos1.index.name].idxmax()]
compos1 = compos1["1998":]
Zscore = (compos1.iloc[minobs1:] - compos1.rolling(minobs1).mean()) / (compos1.rolling(minobs1).std())
Zscore = Zscore.iloc[minobs1:]
RV = -Zscore